And now we're outside the CPU and racing some programs. Read the full story on the official [Day 20 of Advent of Code 2024](https://adventofcode.com/2024/day/20) website.

And the usual ChatGPT generated illustration :

![](chatgpt_image.webp){width=75% fig-align="center"}

::: {.callout-note collapse="true" title="Setting up"}

In [1]:
from misc.helper import verify_answer

example_input_file = "../inputs/example_day_20.txt"
input_file = "../inputs/day_20.txt"

:::

## Part 1: Cheating saves time!

In this task, we are racing a track. We are allowed to cheat -- which means that we can skip one wall.

```
#####
#S# #
# # #
# #E#
#   #
#####
```

In the above example, the shortest path is as follows:

```
#####
#S# #
#|# #
#|#E#
#└-┘#
#####
```

And the cost in this case is `5`. However, if we cheat, we can skip the wall and the cost will be `3`.

<table>
  <tr>
    <td style="text-align: center; vertical-align: middle;">
      <pre>
#####
#S# #
#└-┐#
# #E#
#   #
#####
      </pre>
    </td>
    <td style="text-align: center; vertical-align: middle;">
      <pre>
#####
#S# #
#|# #
#└-E#
#   #
#####
      </pre>
    </td>
    <td style="text-align: center; vertical-align: middle;">
      <pre>
#####
#S-┐#
# #|#
# #E#
#   #
#####
      </pre>
    </td>
  </tr>
</table>


Recipe for Part 1 solution:
- Find the shortest path without cheating.
- Run all possible scenarios where we cheat

In [1]:
from pathlib import Path
import numpy as np


def parse_input(input):
    if Path(input).exists():
        with open(input, "r") as f:
            input = f.read()

    # read the input into the numpy array
    input = input.strip().split("\n")
    grid = np.zeros((len(input), len(input[0])), dtype=int)
    for i, line in enumerate(input):
        for j, char in enumerate(line):
            if char == "#":
                grid[i, j] = 1
            elif char == "S":
                start = (i, j)
            elif char == "E":
                end = (i, j)

    return grid, start, end


example = """
#####
#S# #
# # #
# #E#
#   #
#####
"""

parse_input(example)

(array([[1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 1, 1, 1, 1]]),
 (1, 1),
 (3, 3))

In [11]:
from collections import deque 

def find_shortest_path(grid, start, end):
    n, m = grid.shape

    def bfs(grid, start, end):
        queue = deque([(start, [start])])
        visited = set()

        while queue:
            node, path = queue.popleft()
            if node == end:
                return len(path) - 1

            if node in visited:
                continue

            visited.add(node)
            x, y = node

            for dx, dy in (
                (1, 0),
                (-1, 0),
                (0, 1),
                (0, -1),
            ):
                new_x, new_y = x + dx, y + dy
                if (
                    0 <= new_x < n
                    and 0 <= new_y < m
                    and grid[new_x, new_y] != 1
                ):
                    next_node = (new_x, new_y)
                    if next_node not in visited:
                        queue.append((next_node, path + [next_node]))

        return None

    return bfs(grid, start, end)

find_shortest_path(*parse_input(example))

6

In [12]:
find_shortest_path(*parse_input(example_input_file))

84

In [6]:
def simulate_cheating(grid, start, end):
    initial_cost = find_shortest_path(grid, start, end)
    n, m = grid.shape

    cheaper_paths = {}
    
    def bfs(grid, start, end):
        start_x, strat_y = start
        queue = deque([(start_x, strat_y, False, 0)])
        visited = set()
        visited.add((start_x, strat_y, False))

        while queue:
            x, y, cheated, path_len = queue.popleft()

            if path_len >= initial_cost:
                continue

            if (x, y) == end:
                cheaper_paths[initial_cost - path_len] = cheaper_paths.get(initial_cost - path_len, 0) + 1

            visited.add((x, y, cheated))

            for dx, dy in (
                (1, 0),
                (-1, 0),
                (0, 1),
                (0, -1),
            ):
                new_x, new_y = x + dx, y + dy
                if (
                    0 <= new_x < n
                    and 0 <= new_y < m
                    and grid[new_x, new_y] != 1
                ):
                    if (new_x, new_y, cheated) not in visited:
                        queue.append((new_x, new_y, cheated, path_len + 1))
                elif grid[new_x, new_y] == 1 and not cheated:
                    if (new_x, new_y, True) not in visited:
                        queue.append((new_x, new_y, True, path_len + 1))
                
        return None
    
    bfs(grid, start, end)
    return cheaper_paths

_IncompleteInputError: incomplete input (2778041147.py, line 1)

In [19]:
def simulate_cheating(grid, start, end):

    initial_cost = find_shortest_path(grid, start, end)

    # for each position in the grid that is not on the edge, neither the start nor the end, calculate the cost of the path should it be removed
    n, m = grid.shape
    cheaper_paths = {}
    for i in range(1, n - 1):
        for j in range(1, m - 1):
            if grid[i, j] == 0:
                continue
            grid[i, j] = 0
            cost = find_shortest_path(grid, start, end)
            if cost is not None and cost < initial_cost:
                cheaper_paths[initial_cost - cost] = cheaper_paths.get(initial_cost - cost, 0) + 1
            grid[i, j] = 1


    return cheaper_paths

simulate_cheating(*parse_input(example))

{2: 3}

In [20]:
simulate_cheating(*parse_input(example_input_file))

{4: 14, 12: 3, 2: 14, 10: 2, 8: 4, 6: 2, 64: 1, 20: 1, 40: 1, 38: 1, 36: 1}

In [22]:
example_answer = {
    2: 14,
    4: 14,
    6: 2,
    8: 4,
    10: 2,
    12: 3,
    20: 1,
    36: 1,
    38: 1,
    40: 1,
    64: 1,
}


verify_answer(
    lambda x: simulate_cheating(*parse_input(x)), example_input_file, example_answer
)

✔️ That's right! The answer is {2: 14, 4: 14, 6: 2, 8: 4, 10: 2, 12: 3, 20: 1, 36: 1, 38: 1, 40: 1, 64: 1}.


In [23]:
def part_one(input, saving=0):
    grid, start, end = parse_input(input)
    return sum([v for k, v in simulate_cheating(grid, start, end).items() if k > saving])

part_one(example_input_file, 0 )

44

In [56]:
%time part_one(input_file, 100)

KeyboardInterrupt: 

In [30]:
from pathlib import Path
import numpy as np


def parse_input(input):
    if Path(input).exists():
        with open(input, "r") as f:
            input = f.read()

    # read the input into the numpy array
    input = input.strip().split("\n")
    grid = np.zeros((len(input), len(input[0])), dtype=int)
    walls = set()
    for i, line in enumerate(input):
        for j, char in enumerate(line):
            if char == "#":
                grid[i, j] = 1
                if i < grid.shape[0] - 1 and i > 0 and j < grid.shape[1] - 1 and j > 0:
                    walls.add((i, j))
            elif char == "S":
                start = (i, j)
            elif char == "E":
                end = (i, j)

    return grid, start, end, walls


example = """
#####
#S# #
# # #
# #E#
#   #
#####
"""

parse_input(example)

(array([[1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 1, 1, 1, 1]]),
 (1, 1),
 (3, 3),
 {(1, 2), (2, 2), (3, 2)})

In [66]:
def bfs(grid, target):
    """Calculate distance of each cell from the target"""
    queue = deque([(target, 0)])
    visited = set()

    n, m = grid.shape
    distances = np.full((n, m), np.inf)
    while queue:
        node, distance = queue.popleft()
        if node in visited:
            continue

        visited.add(node)
        x, y = node
        distances[x, y] = distance

        for dx, dy in (
            (1, 0),
            (-1, 0),
            (0, 1),
            (0, -1),
        ):
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < n and 0 <= new_y < m:
                next_node = (new_x, new_y)
                if next_node not in visited:
                    queue.append((next_node, distance + 1))

    return distances


In [65]:
def simulate_cheating(grid, start, end, walls):
    # Precompute distances
    end_distances = bfs(grid, end)
    start_distances = bfs(grid, start)

    initial_cost = find_shortest_path(grid, start, end)

    cheaper_paths = {}
    seen_savings = set()  # Track unique savings

    for wall in walls:
        x, y = wall
        grid[x, y] = 0  # Temporarily remove the wall

        # Calculate new shortest path cost
        new_cost = start_distances[x, y] + end_distances[x, y]

        # Validate that the wall removal creates a valid path
        if new_cost < initial_cost and (x, y) not in seen_savings:
            cheaper_paths[initial_cost - new_cost] = (
                cheaper_paths.get(initial_cost - new_cost, 0) + 1
            )
            seen_savings.add((x, y))  # Track this wall as used for savings

        grid[x, y] = 1  # Restore the wall

    return cheaper_paths


simulate_cheating(*parse_input(example))

{}

In [63]:
def part_one(input, saving=0):
    grid, start, end, walls = parse_input(input)
    return sum([v for k, v in simulate_cheating(grid, start, end, walls).items() if k > saving])
    # return simulate_cheating(grid, start, end, walls)



part_one(example_input_file)

84